In [2]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq


In [31]:
eu_list = pd.read_csv('./data/eu-list.csv', dtype={'CTY_CODE': str})

In [32]:
my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

end_use = "naics?get=CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01"

r = requests.get(url) 
    
print(r)
    
df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

df.columns = r.json()[0]

df["total_imports"] = df["CON_VAL_MO"].astype(float)

df = df[df.SUMMARY_LVL == "DET"]

grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_imports":"sum","CTY_CODE":"first"})

top_products.sort_values(by = "total_imports", ascending = False, inplace = True)


<Response [200]>


In [37]:
not_eu = top_products["CTY_CODE"].isin(eu_list["CTY_CODE"].tolist()) == False

final_list = top_products.loc[not_eu, :].copy(deep=True)

final_list = final_list.drop(index=final_list.index[0])

final_list.reset_index(inplace=True)

In [39]:
final_list.iloc[0:50].to_csv("./data/top_50_non_eu.csv", index=False)

In [38]:
final_list.head()

,CTY_NAME,total_imports,CTY_CODE
0,CHINA,5.728638e+12,5700
1,MEXICO,4.385957e+12,2010
2,CANADA,4.161438e+12,1220
3,JAPAN,1.701399e+12,5880
4,"KOREA, SOUTH",1.045098e+12,5800
